# Bigram

Maximum Likelihood Estimation of n-gram Probability

Context Dependent Smoothing

Witten-Bell Smoothing

In [1]:
#load training data
train_lines = open("data/wiki-en-train.txt", "r").readlines()

#calculate bigram and context counts on training data
counts = {} #bigram and unigram counts
context_counts = {} #bigram context and unigram context counts

for line in train_lines:
    words = line.strip().split(" ")
    words.insert(0, "<s>")
    words.append("</s>")
    for i in range(1, len(words) - 1):
        #count bigram
        if words[i-1] + " " + words[i] in counts.keys():
            counts[words[i-1] + " " + words[i]] += 1
        else:
            counts[words[i-1] + " " + words[i]] = 1

        #count bigram context
        if words[i-1] in context_counts.keys():
            context_counts[words[i-1]] += 1
        else:
            context_counts[words[i-1]] = 1

        #count unigram
        if words[i] in counts.keys():
            counts[words[i]] += 1
        else:
            counts[words[i]] = 1

        #count unigram context
        if "" in context_counts.keys():
            context_counts[""] += 1
        else:
            context_counts[""] = 1

In [2]:
#find context of ngram, calculate the probabilities of ngram, and save the ngram model
bigram_model = open("data/bigram_model.txt", "w")
for ngram, count in counts.items():
    words = ngram.split(" ")
    words.pop()
    context = " ".join(words) #context of ngram
    probability = count / context_counts[context] #calculate MLE of ngram probabilities
    bigram_model.write(ngram + "======>" + str(probability) + "\n") #save the ngram model

In [3]:
#calculate entropy and coverage on the test set using context dependent smoothing

import math

#load ngram model and convert to a dictionary {ngram: probability}
model_bigram = open("data/bigram_model.txt", "r").readlines()
probabilities = {}
for line in model_bigram:
    ngram_p = line.strip().split("======>")
    probabilities[ngram_p[0]] = float(ngram_p[1])

#set parameters
lamda_1 = 0.85 #unknown unigram probability is 1 - lamda_1
lamda_2 = 0.85 #unknown bigram probability is 1 - lamda_2
V = 1000000 #guess total vocabulary size
W_unk = 0 #initiate the counts of unknown ngrams
W = 0 #initiate the counts of all ngrams
H = 0 #initiate the negative log2 likelihood

#load test data
test_lines = open("data/wiki-en-test.txt", "r").readlines()
for line in test_lines:
    words = line.strip().split(" ")
    words.insert(0, "<s>")
    words.append("</s>")
    for i in range(1, len(words) -1):
        if words[i-1] + " " + words[i] not in probabilities.keys():
            W_unk += 1
            #smoothing unigram probability
            if words[i] not in probabilities.keys():
                P1 = (1 - lamda_1) / V 
            else:
                P1 = lamda_1 * probabilities[words[i]] + (1 - lamda_1) / V
            #smoothing bigram probability
            P2 = (1 - lamda_2) * P1
        else:
            #smoothing unigram probability
            if words[i] not in probabilities.keys():
                P1 = (1 - lamda_1) / V 
            else:
                P1 = lamda_1 * probabilities[words[i]] + (1 - lamda_1) / V
            #smooting bigram probability
            P2 = lamda_2 * probabilities[words[i-1]+ " " + words[i]] + (1 - lamda_2) * P1
        H += - (math.log2(P2))
        W += 1
        
print("Entropy =", H / W)
print("Coverage =", (W - W_unk) / W)

Entropy = 12.552196580782692
Coverage = 0.3469903894790086


In [8]:
#function for gram probabilities (lambda) by Witten-Bell Smoothing

def lamda(test_set):
    counts = {} #count ngram
    counts_W = {} #count unigram
    counts_U = {} #count unique ngram
    lamda_dict = {}

    test_lines = open(test_set, "r").readlines()

    for line in test_lines:
        words = line.strip().split(" ")
        words.insert(0, "<s>")
        words.append("</s>")
        for i in range(1, len(words)):
            #check unique ngram
            if words[i-1] + " _" not in counts_U.keys():
                counts_U[words[i-1] + " _"] = 0
            #find and count unigram
            if words[i-1] in counts_W.keys():
                counts_W[words[i-1]] += 1
            else:
                counts_W[words[i-1]] = 1
            #find and count ngram
            if words[i-1] + " " + words[i] in counts.keys():
                counts[words[i-1] + " " + words[i]] += 1
            else:
                counts[words[i-1] + " " + words[i]] = 1
                counts_U[words[i-1] +" _"] += 1 #add unique ngram

    for w, c in counts_W.items():
        lamda_dict[w] = 1 - (counts_U[w + " _"] / (counts_U[w + " _"] + c))

    return lamda_dict

In [9]:
import math

#load bigram model
model_bigram = open("data/bigram_model.txt", "r").readlines()
probabilities = {}
for line in model_bigram:
    ngram_p = line.strip().split("======>")
    probabilities[ngram_p[0]] = float(ngram_p[1])

#set parameters
lamda = lamda("data/wiki-en-test.txt") #gram probabilities (lambda of gram)
V = 1000000 #guess total vocabulary size
W_unk = 0 #initiate the counts of unknown ngrams
W = 0 #initiate the counts of all ngrams
H = 0 #initiate the negative log2 likelihood

#load test data
test_lines = open("data/wiki-en-test.txt", "r").readlines()
for line in test_lines:
    words = line.strip().split(" ")
    words.insert(0, "<s>")
    words.append("</s>")
    for i in range(1, len(words) -1):
        if words[i-1] + " " + words[i] not in probabilities.keys():
            W_unk += 1
            if words[i] not in probabilities.keys():
                P1 = (1 - lamda[words[i]]) / V 
            else:
                P1 = lamda[words[i]] * probabilities[words[i]] + (1 - lamda[words[i]]) / V
            P2 = (1 - lamda[words[i-1]]) * P1
        else:
            if words[i] not in probabilities.keys():
                P1 = (1 - lamda[words[i]]) / V 
            else:
                P1 = lamda[words[i]] * probabilities[words[i]] + (1 - lamda[words[i]]) / V
            P2 = lamda[words[i-1]] * probabilities[words[i-1]+ " " + words[i]] + (1 - lamda[words[i-1]]) * P1 #smoothed bigram probability
        H += - (math.log2(P2))
        W += 1
        
print("Entropy =", H / W)
print("Coverage =", (W - W_unk) / W)

Entropy = 11.652420370041423
Coverage = 0.3469903894790086
